In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Apr 30 17:12:58 2020

@author: hisham
"""

import chardet
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import requests
import json
from flask import jsonify
from csv import writer
from statistics import mean
import itertools
import psycopg2

In [2]:
url = 'http://coronavirus-tracker-api.herokuapp.com/all'
url_iso2 = 'https://restcountries.eu/rest/v2/'


response_iso2 = requests.get(url_iso2)
lst_iso2 = json.loads(response_iso2.text)

response = requests.get(url)
lst = json.loads(response.text)

dict_pop = {}
dict_iso3 = {}
dict_affected = {}
dict_deaths = {}
dict_recovered = {}

for i in lst_iso2:
    dict_pop[i['alpha2Code']] = i['population']
    dict_iso3[i['alpha2Code']] = i['alpha3Code']
    dict_affected[i['alpha2Code']] = {}
    dict_deaths[i['alpha2Code']] = {}
    dict_recovered[i['alpha2Code']] = {}
      

## INITIALIZATION ##

for i in lst['confirmed']['locations']:
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_affected[code][j] = 0

for i in lst['recovered']['locations']:
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_recovered[code][j] = 0

for i in lst['deaths']['locations']:
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_deaths[code][j] = 0         


## Summing up ##

for i in lst['deaths']['locations']:
    name = i['country']
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_deaths[code][j] = dict_deaths[code][j] + i['history'][j] 

for i in lst['recovered']['locations']:
    name = i['country']
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_recovered[code][j] = dict_recovered[code][j] + i['history'][j] 

for i in lst['confirmed']['locations']:
    name = i['country']
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_affected[code][j] = dict_affected[code][j] + i['history'][j]                         



with open('country_model_new.csv', 'w') as csv_file:
    csv_writer = writer(csv_file)
    headers = ['Country_name', 'Country_code_iso2', 'Country_code_iso3', 'Date', 'Tot_affected', 'Tot_dead', 'Tot_recov', 'Population']
    csv_writer.writerow(headers)
    for i in lst['confirmed']['locations']:
        name = i['country']
        code = i['country_code']
        if code in dict_iso3:
            for j in i['history']:
                entry = [name, code, dict_iso3[code], j, dict_affected[code][j], dict_deaths[code][j], dict_recovered[code][j], dict_pop[code]]
                csv_writer.writerow(entry)

In [3]:
with open('country_model_new.csv', 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large

dataset = pd.read_csv('country_model_new.csv', keep_default_na=False,  encoding=result['encoding'])
X = dataset.iloc[:, :].values

#%%
#----------fixing date (Y-m-d)---------
from datetime import datetime as dt, timedelta
for i in range(len(X)):
    # print(i)
    yesterday_str = X[i][3]
    yesterday = dt.strptime(yesterday_str, '%m/%d/%y').date()
    yesterday_str = yesterday.strftime('%Y-%m-%d')
    X[i][3] = yesterday_str

temp1 = np.array(np.ones((len(X),1)))
temp1 = X[:,4] - (X[:,5] + X[:,6])
temp1 = temp1.reshape((len(X),1))
X = np.hstack((X[:, :7], temp1, X[:, 7:]))

#%%
#------delteing extra data----------
# from extra, comes extra new datas -_-
# now let's delete repeating datas
country_data = (X == "Bangladesh").sum()
# country_data = 105

extra = country_data * 230
X = np.delete(X, slice(extra, len(X)), 0)

flag = []
i = 0
while i < len(X):
    j = i+country_data
    while j < len(X):
        if X[i][0] == X[j][0] and flag.count(j) == 0:
            flag.append(j)
        j = j + country_data
    i = i + country_data 
   
flag.reverse()
for i in flag:
    X = np.delete(X, slice(i, i+country_data), 0)
# total country should be 161    
#%%
# 3rd part : calculation
codeList = []
i = 0
while i < len(X):
    #40 threshold set as total cases to avoid unexpected results
    if X[i+country_data-1][4] >= 40 and X[i][0] != "Serbia":
        codeList.append(X[i][1])
    i = i + country_data

# code = codeList[80]     #to test
codeIndex = 1
report = np.empty((0, 8), object)
idx = 0
#--------------looping starts-----------
# code = 'IN'
for code in itertools.islice(codeList, 0, len(codeList)):
    datas = 0
    # arr: country data by dates
    # datas: number of countries
    arr = np.empty((country_data, 9), dtype=object)
    for row in X:
        if row[codeIndex] == code : 
            arr[datas]=row
            datas = datas+ 1
    print(arr[0][0])
     
    arr = arr[np.argsort(arr[:,3])]
    # init: previous values (0 for dead, 1 for affected)
    init = np.empty((datas, 2), dtype=object)
    init[0][0] = arr[0][5]
    init[0][1] = arr[0][4]
    for i in range(1, datas):
        init[i][0] = arr[i][5] - arr[i-1][5]
        init[i][1] = arr[i][4] - arr[i-1][4]
    # for last 2 equations of SIRD
    def calLamda(D):
        lst = []
        for i in range(len(arr) - 1):
            l = 0
            if D == True:
                if arr[i][7] != 0 :     #divide by zero!
                    l = (arr[i+1][5] - arr[i][5]) / arr[i][7]       #lamdaD=(D1 - D0)/ Id0
            else:
                if arr[i][7] != 0 :     #divide by zero!
                    l = (arr[i+1][6] - arr[i][6]) / arr[i][7]       #lamdaR=(R1 - R0)/ Ir0
            if arr[i][3] >= "2020-03-05":                           #timing considered from March 05 to see its growing shape
                lst.append(l)
        return lst
    
    lamdaD = calLamda(True)
    lamdaR = calLamda(False)
    # last 5 values average taken as constant
    ld = mean(lamdaD[-5:])
    # print(ld)
    lr = mean(lamdaR[-5:])
    # print(lr)    
    
    lstBSbyL = []
    for i in range(len(arr) - 1):
        #BS0 = ((Id1 + Ir1 + Id0*lamdaD + Ir0*lamdaR) - (Id0 + Ir0)) / (Id0 + Ir0)
        l = 0
        fplus = arr[i+1][7] + arr[i][7]*(ld + lr)
        splus = arr[i][7]
        if splus != 0:
            l = (fplus - splus)/ splus    
        if arr[i][3] >= "2020-03-05":                           #timing considered from March 05 to see its growing shape
            lstBSbyL.append(l)            
    # last 5 values average taken as constant
    lbsl = mean(lstBSbyL[-5:])
    # print(lbsl/ (ld + lr))
    
    # pop: population of corresponding country
    for i in range(len(X)):
        if X[i][codeIndex] == code:
            pop = X[i][8]
            # print(pop)
            break
    # number of confirmed cases subtracted from actual population
    pop = pop - arr[-1][4]
    # now some hardcoding max, min, average bullshit begins!!
    minConst = 50
    maxConst = 1800
    midConst = 300
    pop1 = pop / minConst
    beta1 = (lbsl / pop1)
    
    pop3 = pop / maxConst
    beta3 = (lbsl / pop3)
    
    pop2 = pop / midConst
    beta2 = (lbsl / pop2)
    
    # here goes our incremental SIRD model; (Eliza, try an ODE version? I haven't tried)
    # this function returns future daily affecteds or deads
    def calcDiff(pop, beta):
        final = np.empty((30, 4), dtype=object)
        final[0][0] = pop - beta*pop*arr[-1][7]                             #S
        final[0][1] = arr[-1][7] + beta*pop*arr[-1][7] - (ld+lr)*arr[-1][7] #I
        final[0][2] = arr[-1][6] + lr*arr[-1][7]                            #R
        final[0][3] = arr[-1][5] + ld*arr[-1][7]                            #D
        
        for i in range(1, len(final)):
            final[i][0] = final[i-1][0] - beta*final[i-1][0]*final[i-1][1]
            final[i][1] = final[i-1][1] + beta*final[i-1][0]*final[i-1][1] - (ld+lr)*final[i-1][1]
            final[i][2] = final[i-1][2] + lr*final[i-1][1]
            final[i][3] = final[i-1][3] + ld*final[i-1][1]
            
        final[:, [0,1,2,3]] = final[:, [0,1,2,3]].astype(int)    
        final = np.append(final, np.ones((len(final),1)), axis = 1)
        final[:,4] = final[:,1] + final[:,2] + final[:,3]                   #cases=active+R+D
        
        diffDead=[]
        diffDead.append(abs(final[0][3] - arr[-1][5]))
        for i in range(1,len(final)):
            diffDead.append(final[i][3] - final[i-1][3])        
        diffAffec=[]
        diffAffec.append(abs(final[0][4] - arr[-1][4]))
        for i in range(1,len(final)):
            diffAffec.append(final[i][4] - final[i-1][4])
       
        return (diffAffec, diffDead)
    
    dA1, dD1 = calcDiff(pop1, beta1)
    dA2, dD2 = calcDiff(pop2, beta2)
    dA3, dD3 = calcDiff(pop3, beta3)
    
    """    
    #ploting stuffs
    # init[:,0]-> dead, 1-> affected; change dA-> for dead
    prevVals = init[:,1]
    x = np.arange(datas, datas+30) 
    plt.plot(prevVals, color = 'b')
    plt.plot(x,dA1, '--', color = 'r')    
    plt.plot(x,dA2, '--', color = 'black')
    plt.plot(x,dA3, '--', color = 'r')
    plt.title(f"{arr[0][0]}")
    plt.show()
    """  
    # now lets create our report, which will be pushed into database
    name = arr[0][0]
    name = np.full((30,1), name)
    code2 = arr[0][1]
    code2 = np.full((30,1), code2)
    
    one_day = timedelta(days = 1)
    yesterday_str = arr[-1][3]
    yesterday = dt.strptime(yesterday_str, '%Y-%m-%d').date()
    Today = np.full((30,1), yesterday_str)
    
    day_predict = []
    for i in range(30):
        day = yesterday + one_day
        yesterday = day
        day = day.strftime("%Y-%m-%d")
        day_predict.append(day)
    day_predict = np.array(day_predict)
    day_predict = day_predict.reshape((30,1))
    
    def createReport (string, v1, v2, v3):
        Type = np.full((30,1), string)        
        v1 = np.array(v1)
        v2 = np.array(v2)
        v3 = np.array(v3)
        v1 = v1.reshape((30,1))
        v2 = v2.reshape((30,1))
        v3 = v3.reshape((30,1))   
        values = np.hstack((Type, name, code2, Today, day_predict, v2, v1, v3))
        values = values.reshape((30, 8))
        return values
    
    reportAffec = createReport("Affected", dA1, dA2, dA3)
    reportDead = createReport("Dead", dD1, dD2, dD3)
    report = np.vstack((report[:idx, :], reportAffec, reportDead, report[idx+60:, :]))
    idx = idx + 60
    
   
    
"""
report
0-> type (affected/ dead)
1-> country_name
2-> code2
3-> database pushing date
4-> prediction date
5-> middle val
6-> max val
7-> min val
"""

Afghanistan
Albania
Algeria
Andorra
Angola
Argentina
Armenia
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Benin
Bolivia
Bosnia and Herzegovina
Brazil
Brunei
Bulgaria
Burkina Faso
Cabo Verde
Cambodia
Cameroon
Canada
Central African Republic
Chad
Chile
China
Colombia
Congo (Brazzaville)
Congo (Kinshasa)
Costa Rica
Cote d'Ivoire
Croatia
Cuba
Cyprus
Czechia
Denmark
Djibouti
Dominican Republic
Ecuador
Egypt
El Salvador
Equatorial Guinea
Estonia
Eswatini
Ethiopia
Finland
France
Gabon
Georgia
Germany
Ghana
Greece
Guatemala
Guinea
Guyana
Haiti
Honduras
Hungary
Iceland
India
Indonesia
Iran
Iraq
Ireland
Israel
Italy
Jamaica
Japan
Jordan
Kazakhstan
Kenya
Korea, South
Kuwait
Kyrgyzstan
Latvia
Lebanon
Liberia
Liechtenstein
Lithuania
Luxembourg
Madagascar
Malaysia
Maldives
Malta
Mauritius
Mexico
Moldova
Monaco
Mongolia
Montenegro
Morocco
Nepal
Netherlands
New Zealand
Niger
Nigeria
North Macedonia
Norway
Oman
Pakistan
Panama
Paraguay
Peru
Philippines
Poland
Portuga

'\nreport\n0-> type (affected/ dead)\n1-> country_name\n2-> code2\n3-> database pushing date\n4-> prediction date\n5-> middle val\n6-> max val\n7-> min val\n'

In [4]:
noCountries = int(len(report)/60)
noCountries




145

In [5]:
report[1][3]


'2020-05-11'

In [6]:
len(report)


8700

In [7]:
def getRegion(regID):
    try:
        

       
    # Print PostgreSQL Connection properties
        print ( connection.get_dsn_parameters(),"\n")

    # Print PostgreSQL version
        cursor.execute("SELECT version();")
        record = cursor.fetchone()
        print("You are connected to - ", record,"\n")
    
        postgres_select_query = "select id from regions where alpha2 = %s"
       
        cursor.execute(postgres_select_query, (regID,))
        
        reg = cursor.fetchall()
        for r in reg :
            print(r[0])
            return r[0]
        #print(reg)

       # connection.commit()
       # count = cursor.rowcount
       # print (count, "Record inserted successfully into worldprediction table")

    except (Exception, psycopg2.Error) as error :
        if(connection):
            print("Error fetching data from PostgreSQL table", error)
    
    #return reg[0]

In [8]:
def insertInfo2(regid,alpha2,typ,ins_date,pred_date,median,maxi,mini,name):
    try:
        postgres_insert_query = """INSERT INTO longtermprediction(region_id,alpha2,type,insertion_date,prediction_date,median,maximum,minimum,name)VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
        cursor.execute(postgres_insert_query, (regid,alpha2,typ,ins_date,pred_date,median,maxi,mini,name))

        connection.commit()
        count = cursor.rowcount
        print (count, "Record inserted successfully into countryprediction table")
        
    except (Exception, psycopg2.Error) as error :
        if(connection):
            print("Error inserting data in PostgreSQL table", error)

In [11]:
import psycopg2
connection = psycopg2.connect(user = "teamcoronosis",
                                  password = "p1jGQwEMANAGZ5b72xWh",
                                  host = "coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com",
                                  port = "5432",
                                  database = "postgres")
cursor = connection.cursor()

#selecting regionIds


        
    #finally:
    #closing database connection.
#inserting into countryprediction table     



#calling functions

regid=[]

for i in range(7395,8700):
    regid=getRegion(report[i][2])
    insertInfo2(regid,report[i][2],report[i][0],report[i][3],report[i][4],report[i][5],report[i][6],report[i][7],report[i][1])
    print(i)

#closing connection
if(connection):
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")


#checking thikmoto insert hoise kina 



{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

202213
1 Record inserted successfully into countryprediction table
7395
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

202213
1 Record inserted successfully into countryprediction table
7396
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p0

1 Record inserted successfully into countryprediction table
7412
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

202213
1 Record inserted successfully into countryprediction table
7413
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

202213
1 Record inserted successfully into countryprediction table
7414
{'user': 't

1 Record inserted successfully into countryprediction table
7430
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

202213
1 Record inserted successfully into countryprediction table
7431
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

202213
1 Record inserted successfully into countryprediction table
7432
{'user': 't

202214
1 Record inserted successfully into countryprediction table
7448
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

202214
1 Record inserted successfully into countryprediction table
7449
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

202214
1 Record inserted successfully into countryprediction table
7450
{'us

1 Record inserted successfully into countryprediction table
7466
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

202214
1 Record inserted successfully into countryprediction table
7467
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

202214
1 Record inserted successfully into countryprediction table
7468
{'user': 't

...

1 Record inserted successfully into countryprediction table
8618
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

202245
1 Record inserted successfully into countryprediction table
8619
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

202245
1 Record inserted successfully into countryprediction table
8620
{'user': 't

1 Record inserted successfully into countryprediction table
8636
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

202245
1 Record inserted successfully into countryprediction table
8637
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

202245
1 Record inserted successfully into countryprediction table
8638
{'user': 't

1 Record inserted successfully into countryprediction table
8654
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

202249
1 Record inserted successfully into countryprediction table
8655
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

202249
1 Record inserted successfully into countryprediction table
8656
{'user': 't

1 Record inserted successfully into countryprediction table
8672
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

202249
1 Record inserted successfully into countryprediction table
8673
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

202249
1 Record inserted successfully into countryprediction table
8674
{'user': 't

1 Record inserted successfully into countryprediction table
8690
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

202249
1 Record inserted successfully into countryprediction table
8691
{'user': 'teamcoronosis', 'dbname': 'postgres', 'host': 'coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 

202249
1 Record inserted successfully into countryprediction table
8692
{'user': 't

In [10]:
if(connection):
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")

PostgreSQL connection is closed
